In [1]:
pip install ripser

Note: you may need to restart the kernel to use updated packages.


In [2]:
# load some packages
from matplotlib.text import Text
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from ripser import ripser
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt

/Users/floyd/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# load the data
df = pd.read_csv(
    'https://www.dropbox.com/scl/fi/1hpq7fymf2syrhwtrb59u/concepts_Applied-Mathematics_102.csv.gz?rlkey=i0s1wyaeww3ngwsc6o9vdwbye&st=920x22gz&raw=1',
    compression='gzip'
)

df


,article_id,year,concept,relevance_mean,concept_freq_in_abstract,concept_no,dfreq_in_category_for_2l,dfreq_in_category_for_2l_year
0,pub.1011927506,2001,abi raising standards scheme,0.0,1,0,1,1
1,pub.1091536887,1988,attu conventional arm control forum,0.0,1,0,1,1
2,pub.1127413625,1997,av 8 harrier,0.0,1,0,1,1
3,pub.1021705670,2010,advertisement game strategy,0.0,1,0,1,1
4,pub.1058195248,1984,aramovich aleksandryan,0.0,1,0,1,1
...,...,...,...,...,...,...,...,...
26399751,pub.1015682740,2008,’s ψ11 summation formula,0.0,1,29,1,1
26399752,pub.1140295407,2021,’ ω transform,0.0,1,4,1,1
26399753,pub.1039011473,2010,’ ϵ variational principle,0.0,1,32,1,1
26399754,pub.1128915375,2020,′ | z | 2 d,0.0,1,16,1,1


In [4]:
# config
MIN_RELEVANCE = 0.7
MIN_YEAR = 1920
MAX_YEAR = 2021  # when the data is from
MIN_CONCEPT_FREQ = 0.0001
MAX_CONCEPT_FREQ = 0.001

# relevance filtering
df = df[df['relevance_mean'] >= MIN_RELEVANCE]

# year filtering
df = df[df['year'] >= MIN_YEAR]

# counts
num_articles = df['article_id'].nunique()
concept_freq = df.groupby('concept').transform('size') / num_articles
df = df[(concept_freq >= MIN_CONCEPT_FREQ) & (concept_freq <= MAX_CONCEPT_FREQ)]

# remove columns we dont care about
df = df[['article_id', 'concept', 'year']]

df

,article_id,concept,year
46607,pub.1110525015,ad hoc networks,2018
46609,pub.1061389704,ad hoc networks,2009
46610,pub.1128617440,ad hoc networks,2020
46611,pub.1103595830,ad hoc networks,2018
46612,pub.1053051088,ad hoc networks,2013
...,...,...,...
26376853,pub.1024144862,zero sum stochastic differential game,2008
26376854,pub.1122600451,zero sum stochastic differential game,2019
26376855,pub.1072010385,zero sum stochastic differential game,2012
26376856,pub.1092410835,zero sum stochastic differential game,2017


In [5]:
# get the initial publication
concepts = (
        df
            .sort_values('year')  # sort so earliest year is first
            .groupby('concept')
            .agg(
                year=('year', 'min'),
                count=('article_id', 'nunique')
            )
            .reset_index()
    )

# normalize the year
concepts['norm_year'] = (concepts['year'] - MIN_YEAR) / (MAX_YEAR - MIN_YEAR)
concepts['inv_count'] = 1 / np.log1p(concepts['count'])

concepts

,concept,year,count,norm_year,inv_count
0,abstract differential equation,1956,32,0.356436,0.286000
1,abstract evolution equations,1972,22,0.514851,0.318929
2,accurate mathematical model,1961,84,0.405941,0.225091
3,accurate numerical solution,1966,24,0.455446,0.310667
4,acoustic emission,2011,27,0.900990,0.300102
...,...,...,...,...,...
4596,zero coupon bond,1995,19,0.742574,0.333808
4597,zero order hold,1970,38,0.495050,0.272958
4598,zero sum differential game,1971,59,0.504950,0.244239
4599,zero sum game,1982,35,0.613861,0.279055


In [6]:
# get all possible edges
edges = df.merge(df, on=['article_id', 'year'], suffixes=['_source', '_target'])

# remove duplicates
edges = edges[edges['concept_source'] < edges['concept_target']]  # remove self links (u - u) and the second occurance (u - v vs v - u)
# edges = edges.sort_values('year').drop_duplicates(subset=['concept_source', 'concept_target']).reset_index(drop=True)
edges = edges.groupby(['concept_source', 'concept_target']).agg(
        year=('year', 'min'),
        count=('article_id', 'nunique')
    ).reset_index()

# nromalize the year
edges['norm_year'] = (edges['year'] - MIN_YEAR) / (MAX_YEAR - MIN_YEAR)
edges['inv_count'] = 1 / np.log1p(edges['count'])

edges

,concept_source,concept_target,year,count,norm_year,inv_count
0,abstract differential equation,approximate controllability,2000,1,0.792079,1.442695
1,abstract differential equation,approximation of solution,1973,1,0.524752,1.442695
2,abstract differential equation,banach space,2016,1,0.950495,1.442695
3,abstract differential equation,boundary control problem,1988,1,0.673267,1.442695
4,abstract differential equation,boundary control system,2013,2,0.920792,0.910239
...,...,...,...,...,...,...
141164,yang mills,yang mills theory,1987,4,0.663366,0.621335
141165,yang mills equation,yang mills theory,1984,4,0.633663,0.621335
141166,zero sum differential game,zero sum game,2018,2,0.970297,0.910239
141167,zero sum differential game,zero sum stochastic differential game,1995,2,0.742574,0.910239


###Processing the Crocker###

In [7]:
# initialize the graph
G = nx.Graph()

# add the nodes
G.add_nodes_from([(c, {'norm_year': ny, 'inv_count': ic}) for c, ny, ic in zip(concepts['concept'], concepts['norm_year'], concepts['inv_count'])])

# add the edges
G.add_edges_from([(u, v, {'norm_year': ny, 'inv_count': ic}) for u, v, ny, ic in zip(edges['concept_source'], edges['concept_target'], edges['norm_year'], edges['inv_count'])])

G

In [8]:
import oat_python as oat

In [9]:
print("norm_year range:", edges['norm_year'].min(), "to", edges['norm_year'].max())
print("inv_count range:", edges['inv_count'].min(), "to", edges['inv_count'].max())

norm_year range: 0.06930693069306931 to 1.0
inv_count range: 0.20511990258137494 to 1.4426950408889634


In [10]:
max_dim = 1
x_vals = np.linspace(0, 1, 100)
betti_curve_diag = np.empty((len(x_vals), max_dim + 1))

adj_year = nx.adjacency_matrix(G, weight='norm_year').tolil()
adj_inv = nx.adjacency_matrix(G, weight='inv_count').tolil()

adj_year.setdiag([d['norm_year'] for _, d in G.nodes(data=True)])
adj_inv.setdiag([d['inv_count'] for _, d in G.nodes(data=True)])

# 遍历每个x值，构建过滤后的图并计算Betti数
for i, x in enumerate(x_vals):
    # 按照两个filtration条件同时过滤
    mask = (adj_year.toarray() <= x) & (adj_inv.toarray() <= x)
    filt_mat = adj_inv.multiply(mask).tocsc()  # 用 inv 作为距离

    # 构造并计算同调
    factored = oat.rust.FactoredBoundaryMatrixVr(filt_mat, max_dim)
    homology = factored.homology(False, False)

    for d in range(max_dim + 1):
        betti_curve_diag[i, d] = np.sum(homology['dimension'] == d)

# 绘图
for d in range(max_dim + 1):
    plt.plot(x_vals, betti_curve_diag[:, d], label=f"$H_{d}$")
plt.xlabel("x (norm_year = norm_inv_count)")
plt.ylabel("Betti number")
plt.title("Betti Curve along norm_year = norm_inv_count")
plt.legend()
plt.grid(True)
plt.show()

thread '<unnamed>' panicked at /Users/runner/.cargo/registry/src/index.crates.io-6f17d22bba15001f/oat_rust-0.1.1/src/topology/simplicial/from/graph_weighted.rs:529:17:


Error: Entry (102,102) of the dissimilarity matrix passed to the `ChainComplexVrFiltered` constructor is Some(OrderedFloat(0.1942692710996454)) but the minimum structural nonzero entry in row 102 is Some(OrderedFloat(0.0)).  In this case `None` represents a value greater strictly greater than `Some(x)`, for every filtration value `x`.
This message is generated by OAT.


note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: 

Error: Entry (102,102) of the dissimilarity matrix passed to the `ChainComplexVrFiltered` constructor is Some(OrderedFloat(0.1942692710996454)) but the minimum structural nonzero entry in row 102 is Some(OrderedFloat(0.0)).  In this case `None` represents a value greater strictly greater than `Some(x)`, for every filtration value `x`.
This message is generated by OAT.

